# SQL Database Agent

This notebook showcases an agent designed to interact with a sql databases. The agent builds off of [SQLDatabaseChain](https://langchain.readthedocs.io/en/latest/modules/chains/examples/sqlite.html) and is designed to answer more general questions about a database, as well as recover from errors.

Note that, as this agent is in active development, all answers might not be correct. Additionally, it is not guaranteed that the agent won't perform DML statements on your database given certain questions. Be careful running it on sensitive data!

This uses the example Chinook database. To set it up follow the instructions on https://database.guide/2-sample-databases-sqlite/, placing the .db file in a notebooks folder at the root of this repository.

### Prepare
1. Pip install Pyodbc
2. Install Microsoft ODBC:
https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=azuresqldb-current&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline#18
3. Setup Azure SQL database
4. Enable Firewall





**Bug Fix for Azure OpenAI**

Got missing 'engine' or 'deployment_name' error when calling completion_with_retry.

langchain/langchina/tools/sql_database/tool.py:6, update line 6 to:
```
from langchain.llms.openai import AzureOpenAI
```


langchain/langchina/tools/sql_database/tool.py:85 change OpenAI to AzureOpenAI
Line 83: 
```
llm_chain: LLMChain = Field(
        default_factory=lambda: LLMChain(
            llm=AzureOpenAI(temperature=0, deployment_name=os.getenv('DEPLOYMENT_NAME')),
            prompt=PromptTemplate(
                template=QUERY_CHECKER, input_variables=["query", "dialect"]
            ),
        )
    )
```


## Initialization

In [1]:
from AzureOpenAIUtil import SqlServer
import os

from dotenv import load_dotenv

load_dotenv()

sql_agent = SqlServer.SqlServer(Server='openai-test.database.windows.net', Database='adventureworks', \
                                Username=os.getenv('AZURE_SQL_USERNAME'), Password=os.getenv('AZURE_SQL_PASSWORD'), topK=20)

## Example: list tables and describing a table

In [2]:
sql_agent.run("How many tables are there?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Artist, Track, Album, PlaylistTrack, Invoice, Playlist, MediaType, Employee, Genre, Customer, InvoiceLine
Thought: I now know the final answer
Final Answer: There are 11 tables in the database.

> Finished chain.


'There are 11 tables in the database.'

In [3]:
sql_agent.run("Describe the playlisttrack table")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Artist, Track, Album, PlaylistTrack, Invoice, Playlist, MediaType, Employee, Genre, Customer, InvoiceLine
Thought: I should look at the schema of the PlaylistTrack table
Action: schema_sql_db
Action Input: "PlaylistTrack"
Observation: 
CREATE TABLE [PlaylistTrack] (
	[PlaylistId] INTEGER NOT NULL, 
	[TrackId] INTEGER NOT NULL, 
	CONSTRAINT [PK_PlaylistTrack] PRIMARY KEY ([PlaylistId], [TrackId]), 
	CONSTRAINT [FK_PlaylistTrackPlaylistId] FOREIGN KEY([PlaylistId]) REFERENCES [Playlist] ([PlaylistId]), 
	CONSTRAINT [FK_PlaylistTrackTrackId] FOREIGN KEY([TrackId]) REFERENCES [Track] ([TrackId])
)

SELECT * FROM 'PlaylistTrack' LIMIT 3;
PlaylistId	TrackId
1	3402
1	3389
1	3390
Thought: I now know the final answer
Final Answer: The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integer values. It is a many-to-many relationship between Playlist and Track.

> Finis

'The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integer values. It is a many-to-many relationship between Playlist and Track.'

## Example: describing a table, recovering from an error

In this example, the agent tries to search for a table that doesn't exist, but finds the next best result

In [4]:
sql_agent.run("Describe the playlistsong table")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Artist, Track, Album, PlaylistTrack, Invoice, Playlist, MediaType, Employee, Genre, Customer, InvoiceLine
Thought: I should look at the schema of the PlaylistSong table
Action: schema_sql_db
Action Input: "PlaylistSong"
Observation: Error: table_names {'PlaylistSong'} not found in database
Thought: I should look at the schema of the PlaylistTrack table
Action: schema_sql_db
Action Input: "PlaylistTrack"
Observation: 
CREATE TABLE [PlaylistTrack] (
	[PlaylistId] INTEGER NOT NULL, 
	[TrackId] INTEGER NOT NULL, 
	CONSTRAINT [PK_PlaylistTrack] PRIMARY KEY ([PlaylistId], [TrackId]), 
	CONSTRAINT [FK_PlaylistTrackPlaylistId] FOREIGN KEY([PlaylistId]) REFERENCES [Playlist] ([PlaylistId]), 
	CONSTRAINT [FK_PlaylistTrackTrackId] FOREIGN KEY([TrackId]) REFERENCES [Track] ([TrackId])
)

SELECT * FROM 'PlaylistTrack' LIMIT 3;
PlaylistId	TrackId
1	3402
1	3389
1	3390
Thought: I now know the final answer


'The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integers. It is a many-to-many relationship between Playlist and Track.'

## Example: running queries

In [5]:
sql_agent.run("List the total sales per country. Which country's customers spent the most?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Artist, Track, Album, PlaylistTrack, Invoice, Playlist, MediaType, Employee, Genre, Customer, InvoiceLine
Thought: I should look at the schema of the Invoice and Customer tables to see what columns I can use.
Action: schema_sql_db
Action Input: "Invoice, Customer"
Observation: 
CREATE TABLE [Customer] (
	[CustomerId] INTEGER NOT NULL, 
	[FirstName] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[LastName] NVARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Company] NVARCHAR(80) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Address] NVARCHAR(70) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[City] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[State] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Country] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[PostalCode] NVARCHAR(10) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Phone] NVA

'Customers from the USA spent the most, with a total of $523.06.'

In [6]:
sql_agent.run("Show the total number of tracks in each playlist. The Playlist name should be included in the result.")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Artist, Track, Album, PlaylistTrack, Invoice, Playlist, MediaType, Employee, Genre, Customer, InvoiceLine
Thought: I should look at the schema of the Playlist and PlaylistTrack tables to see what columns I can use.
Action: schema_sql_db
Action Input: "Playlist, PlaylistTrack"
Observation: 
CREATE TABLE [Playlist] (
	[PlaylistId] INTEGER NOT NULL, 
	[Name] NVARCHAR(120) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK_Playlist] PRIMARY KEY ([PlaylistId])
)

SELECT * FROM 'Playlist' LIMIT 3;
PlaylistId	Name
1	Music
2	Movies
3	TV Shows


CREATE TABLE [PlaylistTrack] (
	[PlaylistId] INTEGER NOT NULL, 
	[TrackId] INTEGER NOT NULL, 
	CONSTRAINT [PK_PlaylistTrack] PRIMARY KEY ([PlaylistId], [TrackId]), 
	CONSTRAINT [FK_PlaylistTrackPlaylistId] FOREIGN KEY([PlaylistId]) REFERENCES [Playlist] ([PlaylistId]), 
	CONSTRAINT [FK_PlaylistTrackTrackId] FOREIGN KEY([TrackId]) REFERENCES [Track] 

'The total number of tracks in each playlist is: 90’s Music (1477), Brazilian Music (39), Classical (75), Classical 101 - Deep Cuts (25), Classical 101 - Next Steps (25), Classical 101 - The Basics (25), Grunge (15), Heavy Metal Classic (26), Music (6580), Music Videos (1), On-The-Go 1 (1), TV Shows (426).'